```
This file is part of Estimation of Causal Effects in the Alzheimer's Continuum (Causal-AD).

Causal-AD is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

Causal-AD is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with Causal-AD. If not, see <https://www.gnu.org/licenses/>.
```

# Fit Beta Regression Models

ADAS-Cog-13 has a maximum of 85.

See [Supplement](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3732822/bin/NIHMS420697-supplement-Supplementary_Table_1.docx) of https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3732822/

## Parameters

In [ ]:
# define and set the parameters for papermill
input_csv <- "r_data.csv"
coef_output_file <- "coef.csv"
rng_seed <- 2501

In [ ]:
if (file.exists(coef_output_file)) error(paste("file", coef_output_file, "already exists."))

## Helper Functions

In [ ]:
suppressMessages(library(bayesplot))
suppressMessages(library(rstanarm))
library(dplyr)
library(ggplot2)

options(mc.cores = 2)

In [ ]:
load_data <- function(filename) {
    data <- read.csv(filename, row.names=1)

    outcome <- data[, "ADAS13", drop=FALSE]

    # Convert ADAS13 to proportions.
    max.ADAS13 <- 85
    if(any(outcome$ADAS13 >= max.ADAS13)) error("ADAS13 score too big")
    if(any(outcome$ADAS13 <= 0)) error("ADAS13 score too small")
    outcome$ADAS13 <- outcome$ADAS13 / 85

    cnames <- colnames(data)
    if (length(unique(cnames)) != length(cnames)) { error("column names are not unique") }
    last.col <- which(cnames == "ADAS13")
    if (last.col < 2) { error("features must come before ADAS13 column") }
    resid.cols <- seq(1, last.col - 1)

    resid.data <- cbind(data[, resid.cols], outcome)

    return(resid.data)
}

In [ ]:
fit_betareg <- function(data) {
    mm <- terms(as.formula(ADAS13 ~ .), data=data)
    frm <- paste("ADAS13 ~", paste(attr(mm, "term.labels"), collapse = "+"))
    cat(frm)
    f <- as.formula(frm)
    f <- stan_betareg(
        f,
        data = data,
        link = "logit",
        prior = normal(location = 0, scale = 2),
        prior_intercept = normal(location = 0, scale = 10),
        prior_phi = cauchy(location = 0, scale = 5),
        algorithm = "sampling",
        iter = 5000,
        seed = rng_seed,
        chains = 4,
        na.action=na.fail,
    )
    return(f)
}

train_test_split <- function(indices) {
    n_samples <- length(indices)
    idx <- sample(1:n_samples)
    num_test <- as.integer(0.1 * n_samples)
    test_idx <- indices[idx[1:num_test]]
    train_idx <- indices[idx[(num_test + 1):n_samples]]
    return(list(train.idx=train_idx, test.idx=test_idx))
}

## Fit Model

In [ ]:
resid.data <- load_data(input_csv)

dim(resid.data)

Split into train and test.

In [ ]:
set.seed(rng_seed)
splits <- train_test_split(rownames(resid.data))

In [ ]:
fit <- fit_betareg(resid.data[splits$train.idx,])

## PPC

In [ ]:
ppc_stats_with_pval <- function(y, yrep, stat) {
    fn <- match.fun(stat)
    pb <- mean(apply(yrep, 1, fn) >= fn(y))
    pb <- round(pb, 3)
    p <- ppc_stat(
        y,
        yrep,
        stat=stat
    ) +
#     ggtitle(as.expression(substitute(
#         paste(italic(T), "=", stat)
#     ))) +
    labs(subtitle = as.expression(substitute(
        paste(italic(P), "(",
              italic(T), "(", italic("y")["rep"], ")≥",
              italic(T), "(", italic("y"), "))",
              " = ", pb))
    ))
    return(p)
}

ppc_grid <- function(data.true, data.rep) {
    p.1 <- ppc_stats_with_pval(
        data.true,
        data.rep,
        stat="mean"
    )

    p.2 <- ppc_stats_with_pval(
        data.true,
        data.rep,
        stat="min"
    )

    p.3 <- ppc_stats_with_pval(
        data.true,
        data.rep,
        stat="max"
    )

    bayesplot_grid(
       p.1, p.2, p.3, legends=TRUE
    )
}

In [ ]:
ppc_grid(resid.data[splits$train, "ADAS13"], posterior_predict(fit))

In [ ]:
adas.rep <- posterior_predict(fit, newdata=resid.data[splits$test,])

ppc_grid(resid.data[splits$test, "ADAS13"], adas.rep)

ppc_dens_overlay(resid.data[splits$test, "ADAS13"], adas.rep)

## Write coefficients

In [ ]:
write.csv(fit$stan_summary, file=coef_output_file)

In [ ]:
write.csv(
    as.matrix(fit$stanfit),
    paste(c(dirname(coef_output_file), paste0("samples_", basename(coef_output_file))), collapse="/"),
    row.names=FALSE,
)

In [ ]:
write.csv(
    data.frame(test.idx=splits$test.idx),
    paste(c(dirname(coef_output_file), paste0("test-idx_", basename(coef_output_file))), collapse="/"),
    row.names=FALSE,
)